In [4]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

In [6]:
aq.Plan.last()[0]

{'budget_id': 77,
 'user_id': 209,
 'created_at': '2021-04-27T10:01:22.000-07:00',
 'rid': 167,
 'status': None,
 'updated_at': '2021-04-27T10:05:11.000-07:00',
 'folder': None,
 'name': 'Xinru_a5b1_sorted_library_NGSprep',
 'id': 41306,
 'cost_limit': None,
 'layout': '{"id": 0, "parent_id": -1, "name": "no_name", "width": 160, "height": 60, "input": null, "output": null, "documentation": "No documentation for this module", "children": null, "wires": null, "text_boxes": null}'}

# Count and Bin Yeast Strains

In [61]:
yeast_type = aq.SampleType.find_by_name("Yeast Strain")

samples = aq.Sample.where({'sample_type_id': yeast_type.id, 'user_id': 66})


filtered_samples = []
for s in samples:
    if arrow.get(s.created_at) > arrow.Arrow(year=2018, month=6, day=1):
        filtered_samples.append(s)

all_items = []

for otname in ['Yeast Glycerol Stock', 'Yeast Plate']:
    ot = aq.ObjectType.find_by_name(otname)
    items = aq.Item.where({'sample_id': [s.id for s in filtered_samples], 'object_type_id': ot.id})
    all_items.extend(items)

In [62]:
unique_ids = list(set([item.sample_id for item in all_items]))

len(unique_ids)

431

In [63]:
import arrow

times = []

def pairwise(x):
    for a, b in zip(x[:-1], x[1:]):
        yield a, b

bins = []
quarters = []

for year in [2018, 2019, 2020, 2021]:
    for month in [1, 4, 7, 10]:
        quarters.append((month, year))
        
for month, year in quarters:
    times.append(arrow.Arrow(year=year, month=month, day=1))
    bins.append([])
bins = bins[:-1]

groupby_sample = {}

for item in all_items:
    groupby_sample.setdefault(item.sample_id, list())
    groupby_sample[item.sample_id].append(item)
    
groupby_sample = {k: sorted(items, key=lambda x: arrow.get(item.created_at)) for k, items in groupby_sample.items()}

for sid, items in groupby_sample.items():
    for item in items:
        for i, (t1, t2) in enumerate(pairwise(times)):
            t3 = arrow.get(item.created_at)
            if t1 < t3 < t2:
                bins[i].append(item)
            
bin_sids = [list(set([i.sample_id for i in b])) for b in bins]

for q, b in zip(quarters, bin_sids):
    print(q, len(b))

(1, 2018) 0
(4, 2018) 7
(7, 2018) 77
(10, 2018) 132
(1, 2019) 100
(4, 2019) 75
(7, 2019) 0
(10, 2019) 2
(1, 2020) 113
(4, 2020) 0
(7, 2020) 0
(10, 2020) 5
(1, 2021) 28
(4, 2021) 0
(7, 2021) 0


In [58]:
for q, b in zip(quarters, bin_sids):
    print(q, len(b))

(1, 2018) 3
(4, 2018) 10
(7, 2018) 70
(10, 2018) 134
(1, 2019) 94
(4, 2019) 78
(7, 2019) 0
(10, 2019) 4
(1, 2020) 112
(4, 2020) 0
(7, 2020) 0
(10, 2020) 5
(1, 2021) 28
(4, 2021) 0
(7, 2021) 0


In [64]:
import pandas as pd

pd.DataFrame([len(b) for b in bins]).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,18,148,150,123,126,0,2,209,0,0,13,55,0,0


# Count and Bin Genetic Parts

In [11]:
import arrow

sample_type = aq.SampleType.find_by_name("Yeast Strain")

samples = aq.Sample.where({'sample_type_id': sample_type.id, 'user_id': 66})


filtered_samples = []
for s in samples:
    if arrow.get(s.created_at) > arrow.Arrow(year=2018, month=6, day=1):
        filtered_samples.append(s)

all_items = []

with aq.with_cache(timeout=67) as sess:
    for otname in ['Yeast Plate']:
        ot = sess.ObjectType.find_by_name(otname)
        items = sess.Item.where({'sample_id': [s.id for s in filtered_samples], 'object_type_id': ot.id})
        all_items.extend(items)

In [12]:
import arrow
from copy import deepcopy

times = []

def pairwise(x):
    for a, b in zip(x[:-1], x[1:]):
        yield a, b

bins = []
quarters = []

for year in [2018, 2019, 2020, 2021]:
    for month in [1, 4, 7, 10]:
        quarters.append((month, year))
        
for month, year in quarters:
    times.append(arrow.Arrow(year=year, month=month, day=1))
    bins.append([])
    
bins = bins[:-1]

groupby_sample = {}

for item in all_items:
    groupby_sample.setdefault(item.sample_id, list())
    groupby_sample[item.sample_id].append(item)
    
groupby_sample = {k: sorted(items, key=lambda x: arrow.get(item.created_at)) for k, items in groupby_sample.items()}

first_item_bins = deepcopy(bins)

for sid, items in groupby_sample.items():
    first_item = items[0]
    
    for item in items[:1]:
        for i, (t1, t2) in enumerate(pairwise(times)):
            t3 = arrow.get(item.created_at)
            if t1 < t3 < t2:
                first_item_bins[i].append(item)
                
    for item in items:
        for i, (t1, t2) in enumerate(pairwise(times)):
            t3 = arrow.get(item.created_at)
            if t1 < t3 < t2:
                bins[i].append(item)
            
bin_sids = [list(set([i.sample_id for i in b])) for b in first_item_bins]

for q, b in zip(quarters, bin_sids):
    print(q, len(b))
    
    
import pandas as pd

pd.DataFrame([len(b) for b in bins]).T

(1, 2018) 0
(4, 2018) 7
(7, 2018) 53
(10, 2018) 56
(1, 2019) 55
(4, 2019) 23
(7, 2019) 0
(10, 2019) 0
(1, 2020) 112
(4, 2020) 0
(7, 2020) 0
(10, 2020) 4
(1, 2021) 18
(4, 2021) 0
(7, 2021) 0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,17,79,72,73,50,0,2,146,0,0,8,35,0,0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,7,53,56,55,23,0,0,112,0,0,4,18,0,0
